In [ ]:
pip install numpy scikit-learn faiss-cpu

In [ ]:
#Real-time Context Retrieval System in Google Colab

# Step 1: Set up the environment
!pip install faiss-cpu

# Step 2: Import required libraries
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import faiss

# Step 3: Define the RealtimeContextRetrieval class
class RealtimeContextRetrieval:
    def __init__(self, documents):
        self.documents = documents
        self.vectorizer = TfidfVectorizer()
        self.document_vectors = self.vectorizer.fit_transform(documents)

        # Convert sparse matrix to dense numpy array
        self.document_vectors_dense = self.document_vectors.toarray().astype('float32')

        # Create a FAISS index
        self.dimension = self.document_vectors_dense.shape[1]
        self.index = faiss.IndexFlatL2(self.dimension)
        self.index.add(self.document_vectors_dense)

    def retrieve_context(self, query, k=5):
        query_vector = self.vectorizer.transform([query]).toarray().astype('float32')

        # Search the FAISS index
        distances, indices = self.index.search(query_vector, k)

        results = []
        for i, idx in enumerate(indices[0]):
            results.append({
                'document': self.documents[idx],
                'similarity': 1 - distances[0][i] / 2  # Convert L2 distance to similarity
            })

        return results

    def add_document(self, new_document):
        self.documents.append(new_document)
        new_vector = self.vectorizer.transform([new_document]).toarray().astype('float32')
        self.index.add(new_vector)

# Step 4: Initialize the system with sample documents
documents = [
    "Python is a high-level programming language.",
    "Machine learning is a subset of artificial intelligence.",
    "Natural language processing deals with interactions between computers and human language.",
    "Deep learning is part of a broader family of machine learning methods.",
    "Transformer models have revolutionized natural language processing tasks."
]

retrieval_system = RealtimeContextRetrieval(documents)

# Step 5: Test the system with a query
query = "What is machine learning?"
results = retrieval_system.retrieve_context(query)

print(f"Query: {query}")
print("Top 5 relevant contexts:")
for i, result in enumerate(results, 1):
    print(f"{i}. {result['document']} (Similarity: {result['similarity']:.4f})")

# Step 6: Add a new document and test again
new_document = "Reinforcement learning is an area of machine learning concerned with how agents take actions in an environment."
retrieval_system.add_document(new_document)

results = retrieval_system.retrieve_context(query)

print("\nAfter adding a new document:")
print(f"Query: {query}")
print("Top 5 relevant contexts:")
for i, result in enumerate(results, 1):
    print(f"{i}. {result['document']} (Similarity: {result['similarity']:.4f})")

# Step 7: Interactive demo
from IPython.display import clear_output

def interactive_demo():
    while True:
        clear_output(wait=True)
        print("Real-time Context Retrieval System")
        print("1. Retrieve context")
        print("2. Add new document")
        print("3. Exit")
        choice = input("Enter your choice (1/2/3): ")

        if choice == '1':
            query = input("Enter your query: ")
            results = retrieval_system.retrieve_context(query)
            print(f"\nQuery: {query}")
            print("Top 5 relevant contexts:")
            for i, result in enumerate(results, 1):
                print(f"{i}. {result['document']} (Similarity: {result['similarity']:.4f})")
        elif choice == '2':
            new_doc = input("Enter new document: ")
            retrieval_system.add_document(new_doc)
            print("Document added successfully!")
        elif choice == '3':
            print("Thank you for using the Real-time Context Retrieval System!")
            break
        else:
            print("Invalid choice. Please try again.")

        input("\nPress Enter to continue...")

# Run the interactive demo
interactive_demo()